In [3]:
import pprint
from ollama import chat
from fastmcp import Client
from fastmcp.client.transports import SSETransport

In [4]:
transport = SSETransport(url="http://localhost:8082/sse")
async with Client(transport) as client:
    tools = await client.list_tools()
    ollama_tools = []
    for tool in tools:
        tool_dict = tool.model_dump()

        ollama_tools.append({
            "type": "function",
            "function": {
                "name": tool_dict["name"],
                "description": tool_dict["description"],
                "parameters": tool_dict["inputSchema"]
            }
        })

In [16]:
content = """
당신은 최신 정보를 찾는 AI 리서처입니다.

1. 사용자의 질문을 의미 있는 키워드 또는 주제별로 분해하세요.
2. 각 키워드/주제마다 3~5개의 웹검색 쿼리를 생성하세요.
3. 각 쿼리에 대해 info_search_web(또는 search_web) 툴을 호출해 최신 정보를 수집하세요.
4. 수집한 정보를 통합하여, 신뢰할 수 있는 출처(URL)와 함께 요약 보고서를 작성하세요.
5. 요약은 명확하고, 사실 기반이며, 출처를 반드시 포함하세요.

예시 입력:
"2024년 미국 대선 후보의 주요 공약과 최근 AI 규제 동향을 알려줘"

예시 작업 흐름:
- [1단계] 쿼리 분해: ["2024년 미국 대선 후보 주요 공약", "2024년 미국 AI 규제 동향"]
- [2단계] 각 항목별 검색 쿼리 생성
- [3단계] 각 쿼리로 info_search_web 툴 콜
- [4단계] 결과 통합 및 출처 포함 요약

각 툴 콜의 파라미터 예시:
{
  "query": "2024 US presidential candidates main pledges",
  "date_range": "past_month"
}

{
  "query": "2024 US AI regulation trends",
  "date_range": "past_month"
}
"""

messages = [
    {"role": "system", "content": content},
    {"role": "user", "content": "주술회전, 나루토, 원피스, 김문수 가 뭐야? /no_think"}
]

print("STEP 1: Call Tools")
response = chat(
    model="qwen3:8b",
    messages=messages,
    tools=ollama_tools,
    keep_alive = -1
)

if "tool_calls" in response["message"]:
    pprint.pprint(response["message"]["tool_calls"])
else:
    print("툴 없음")
    pprint.pprint(response["message"]["content"])

STEP 1: Call Tools
[ToolCall(function=Function(name='web_search', arguments={'query': '주술회전 나루토 원피스 김문수 뜻'})),
 ToolCall(function=Function(name='web_search', arguments={'query': '주술회전 나루토 원피스 김문수 정의'})),
 ToolCall(function=Function(name='web_search', arguments={'query': '주술회전 나루토 원피스 김문수 설명'}))]


In [4]:
if "tool_calls" in response["message"]:
    tool_calls = response["message"]["tool_calls"]
    for tool_call in tool_calls:
        fn = tool_call["function"]
        async with Client(transport) as mcp_client:
            result = await mcp_client.call_tool(
                name = fn["name"], 
                arguments = fn["arguments"]
            )

        messages.append({
                    "role": "tool",
                    "name": fn["name"],
                    "content": result[0].text + " /no_think"
                })
    
print("STEP 2: Add Tool Result")
pprint.pprint(messages)
print()

STEP 2: Add Tool Result
[{'content': 'You are a helpful assistant. Use the toolㄴㄴ for questions.',
  'role': 'system'},
 {'content': '주술회전이 뭐야? /no_think', 'role': 'user'},
 {'content': '주술회전\n'
             '-\n'
             '나무위키\n'
             '최근\n'
             '변경최근\n'
             '토론특수\n'
             '기능\n'
             '주술회전\n'
             '최근\n'
             '수정\n'
             '시각:\n'
             '2025-05-05\n'
             '07:26:07138편집\n'
             '요청편집\n'
             '요청편집\n'
             '권한이\n'
             '부족합니다.\n'
             '로그인된\n'
             '사용자(이)여야\n'
             '합니다.\n'
             '해당\n'
             '문서의\n'
             'ACL\n'
             '탭을\n'
             '확인하시기\n'
             '바랍니다.\n'
             '편집\n'
             '권한이\n'
             '부족한\n'
             '경우\n'
             '아래의\n'
             "'편집\n"
             "요청'\n"
             '버튼으로\n'
             '편집\n'
             '요청을\n'
             '생성할\n'
             '수\n'
   

In [5]:
response = chat(
    model="qwen3:8b",
    messages=messages,
    tools=ollama_tools,
    keep_alive = -1
)

print("STEP 3: Final Answer")
pprint.pprint(response["message"]["content"])

STEP 3: Final Answer
('<think>\n'
 '\n'
 '</think>\n'
 '\n'
 "It looks like you've pasted a long block of text that seems to be a mix of "
 'content from a Korean webtoon or manga, possibly **"Jujutsu Kaisen" '
 '(咒术回战)**, along with some metadata, footnotes, and possibly some unrelated '
 "text or formatting issues. Here's a breakdown of what I see:\n"
 '\n'
 '---\n'
 '\n'
 '### 📖 **Main Content: "Jujutsu Kaisen" (咒术回战)**\n'
 '\n'
 'This appears to be a **summary or analysis** of the manga *Jujutsu Kaisen* '
 '(which translates to "Jujutsu Battle") by **Gege Akutami**. It includes:\n'
 '\n'
 '- **Characters** like:\n'
 '  - **Gojo Satoru**\n'
 '  - **Megumi Fushiguro**\n'
 '  - **Toge Inuishi**\n'
 '  - **Tsuji Satoru**\n'
 '  - **Haguro Hiroto**\n'
 '- **Plot points**:\n'
 '  - The **"Satoru Gojo" arc**\n'
 '  - The **"Shinigami" arc**\n'
 '  - **Kasugano** and the **Cursed Techniques**\n'
 '  - **Cursed Wands**, **Cursed Spirits**, and **Cursed Techniques**\n'
 '- **Themes**:\n'
 ' 